Goal
- Find the smallest eigenvector $\lambda$ for the NC node

Problem
- Some methods too slow and/or crash on certain cases
---
Input
 - Positive definite matrix (symmetric)

Output
 - Graph cut of the matrix (corresponds to the smallest eigenvector)
---
Must
- Handle image sized inputs
- Have reproducible results (fixed seed)

In [14]:
import numpy as np
from scipy import linalg
import torch
## TODO: Begin with a simple 'image' that has a known cut, this way it can be tested if correct
## May be able to compare to sklearn.cluster.SpectralClustering (https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html?highlight=lobpcg#r5f6cbeb1558e-4)

seed = 123
dtype = np.float64 # Currently doesn't do anything as random.rand doesn't accept
n = 50
# and possibly generate different sparisities?? As realistically will be somewhat sparse
np.random.seed(seed)



# Random image of size
input = np.random.randint(0,255, n*n)

# TODO: Test for larger sized (full correctly formed, symmetric pos def - but random, and fully random matricies)
# TODO: Test against multiple different inputs (list of inputs) and also plot the times, and outputs of each to see if they are correct / consistent

# pretend simple image
# input = np.array([1,1,1,1,255,255,255,255,255,255,
#                   1,1,1,1,255,255,255,255,255,255,
#                   1,1,1,1,255,255,255,255,255,255,
#                   1,1,1,1,255,255,255,255,255,255,
#                   1,1,1,1,255,255,255,255,255,255,
#                   1,1,1,1,255,255,255,255,255,255,
#                   1,1,1,1,1,255,255,255,255,255,
#                   1,1,1,1,1,255,255,255,255,255,
#                   1,1,1,1,1,1,255,255,255,255,
#                   1,1,1,1,1,1,1,1,255,255,])

# in = n random numbers between 0 and 255 # probably better if its slightly realistic?
A = linalg.fiedler(input)

# TODO: Change this with a real image, and real values from it... 
#       but for now should be fine (same properties being symmetric positive semi-definite)

A = torch.from_numpy(A)
A = A.reshape(1,n*n,n*n)

# print(A)
print(input.shape)
print(A.shape)

(2500,)
torch.Size([1, 2500, 2500])


In [15]:
# Similar to https://gist.github.com/denis-bz/6a9d7379c8edf965b0a997c2ec2471e1

# used to store each of the functions, and allow them to all take the single arg input
from collections import OrderedDict
from functools import partial

# scipy and numpy are used for the eigensolvers
import scipy

# TODO: reinstall scikit-sprase on mac to get it to install properly (wrong depecencies and unknown fix for mac)
# until then just dont test unless on debian system
sksparse_present = True
try:
    import sksparse 
except:
    sksparse_present = False


# TODO: use the initial vector :)
# TODO: and use it such that you test both v0 set to 0 and v0 set to random :) to see if any differences
v0 = np.zeros_like(A) # initialise the initial vector to all zeros :)
# TODO: Switch v0 to something like X = rng.normal(size=(n, 3)) so normally distributed around origin


rcond = 1e-6 # cutoff for small singular values of a (0 if smaller than rcond * largest singular of a)

d = A.sum(1) # 1 because 0 is batch
D = torch.diag_embed(d) # D = matrix with d on diagonal
D = D[0] # Takes form b,N*N,N*N but needs to be non batch for non batch eigensolvers


eigs_options = OrderedDict(
    ## NOTE: Will need to set the v0 to something consistent
    ## NOTE: and if neccessary any seeds used by them....
    ## NOTE: All inputs will be positive definite so should be easy :)
    
    
    # Types to try:
    # - shift invert (as we are looking for smallest) (https://gist.github.com/denis-bz/2658f671cee9396ac15cfe07dcc6657d)
    # - Power iteration, QR, LOBPCG
    # - Lanzcos, Arnoldi
    # - cholmod (https://scikit-sparse.readthedocs.io/en/latest/cholmod.html, https://stackoverflow.com/questions/59416098/finding-smallest-eigenvectors-of-large-sparse-matrix-over-100x-slower-in-scipy)
    # - any gpu based ones? (pytorch perhaps?)

    # Numpy (no params only inputs)
    np_eig = np.linalg.eig,
    np_eigh = np.linalg.eigh, # Should be good
    np_eigvals = np.linalg.eigvals,
    
    # Some parameters for scipy variants
    sp_eig = partial(scipy.linalg.eig, check_finite=False), # No extra params

    sp_eigh_base = partial(scipy.linalg.eigh), # Should be good (and all variations might improve for the single eigenvector case)
    sp_eigh_cff = partial(scipy.linalg.eigh, check_finite=False),
    # Subset by index only for evr, evx, and gvx
    # Subsetting (0,1] so that [1] gives the second smallest (so indexing is the same as others which return all vectors)
    # [1,1] will give just the second smallest
    sp_eigh_ss = partial(scipy.linalg.eigh, check_finite=False, subset_by_index=[0,1]), # driver=, type=(generalized or not), 
    
    sp_eigh_ev_ss = partial(scipy.linalg.eigh, check_finite=False, driver='ev'), # symmetric qr, slow but robust
    sp_eigh_evd = partial(scipy.linalg.eigh, check_finite=False, driver='evd'), # uses more memory but faster
    sp_eigh_evr_ss = partial(scipy.linalg.eigh, check_finite=False, subset_by_index=[0,1], driver='evr'), # optimal for most
    sp_eigh_evx_ss = partial(scipy.linalg.eigh, check_finite=False, subset_by_index=[0,1], driver='evx'), # subsets
    
    # uses D for the generalized problem
    sp_eigh_gv = partial(scipy.linalg.eigh, check_finite=False, driver='gv', b=D),
    sp_eigh_gvd = partial(scipy.linalg.eigh, check_finite=False, driver='gvd', b=D),
    sp_eigh_gvx_ss = partial(scipy.linalg.eigh, check_finite=False, subset_by_index=[0,1], driver='gvx', b=D),
    
    
    
    # Sparse boys, often more complex may include a v0 and maxiters etc
    # use 
    # TODO: based on the gist implement fancy variations of these (inclue differences in max iters etc..)
    # TODO: and utuilise sigma to to the inverted faster thing :)
    # sp_sparse_eigs = partial(scipy.sparse.linalg.eigs, k=1),   #  maxiter=max_iter, tol=0, which='SM', k=1)
    # sp_sparse_eigsh = partial(scipy.sparse.linalg.eigsh, k=1),
    
    # Slightly less useful ones?
    sp_sparse_lobpcg = partial(scipy.sparse.linalg.lobpcg(), X=v0, largest=False, maxiter=40), # TODO: Verify if n, works instead of n,1
    sp_sparse_lobpcg_2 = partial(scipy.sparse.linalg.lobpcg(), X=v0, B=D, largest=False, maxiter=40), # TODO: Could test using constraints, maxiter, tol
    sp_sparse_bicg = partial(scipy.sparse.linalg.bicg(), b=D), # or bicgstab? TODO: add params for x0, tol, atol, maxiter maybe
    sp_sparse_gmres = partial(scipy.sparse.linalg.gmres(), b=D), # TODO: add params for x0, tol, atol, maxiter maybe
    # sp_sparse_splu = partial(scipy.sparse.linalg.splu()), # Only works on sparse matricies
    
    np_qr = np.linalg.qr(), # TODO: Idk if the outputs of this will match the others... maybe test by hand
    np_cholesky = np.linalg.cholesky(), # Outputs only one value, so should work?
    # sp_qr = partial(scipy.linalg.qr(), check_finite=False), # TODO: Check outputs of this by hand and if they match
    sp_cholesky = partial(scipy.linalg.cholesky(), check_finite=False),

    # Weirdo Ones
    # TODO: they will return the wrong order of stuff.. so will manually check and test?
    # np_lstsq = partial( np.linalg.lstsq, b=D, rcond=rcond ),
    # sp_lstsq = partial( scipy.linalg.lstsq, b=D, cond=rcond ),
    # np_solve = partial( np.linalg.solve, b=D ),
    # sp_solve = partial( scipy.linalg.solve, b=D ),
    # np_svd = partial( np.linalg.svd, compute_uv=False ),  # gesdd
    # sp_svd = partial( scipy.linalg.svd, compute_uv=False ),  # lapack_driver : {'gesdd', 'gesvd'}
    
    
)

# scikit-sparse doesn't install nicely on MacOS (but does nicely on debian) so may not be present to test
if sksparse_present:
    eigs_options['sks_cholmod_cholesky'] = sksparse.cholmod.cholesky # Should be good? (TODO: Verify general eigen vs none)
        

In [16]:
from time import time
import sys
sys.path.append("../")
from nc import NormalizedCuts

node = NormalizedCuts(eps=1e-8)#, bipart=args.bipart, symm_norm_L=args.symm_norm_L)



for name, func in eigs_options.items(): 
    t0 = time()
    y,_ = node.solve(A,func=func) # The output also includes context (not needed herex)
    t = time() - t0
    
    y = torch.real(y)
    y = y.reshape(1,n,n)
    
    
    obj = node.objective(A,y)
    eqconst = node.equality_constraints(A,y)
    # Check against objetive function, should solve as close to machine precision as possible    
    print(f"{name:20}: {t:5.0f} sec | \tsolution = {obj.item():1.5f} | \teqconst = {eqconst.item():1.5f}")

np_eig              :    11 sec | 	solution = 0.00000 | 	eqconst = -528819378.00000
np_eigh             :     3 sec | 	solution = 1.00000 | 	eqconst = 0.00000
np_eigvals          :     6 sec | 	solution = 0.05049 | 	eqconst = -5598531181924992.00000
scipy_eig           :    11 sec | 	solution = -0.00000 | 	eqconst = -528819378.00000
scipy_eigh_base     :    12 sec | 	solution = 1.00000 | 	eqconst = 0.00000
scipy_eigh_cff      :    11 sec | 	solution = 1.00000 | 	eqconst = 0.00000
scipy_eigh_ss       :     2 sec | 	solution = 1.00000 | 	eqconst = 0.00000
scipy_eigh_ev_ss    :    15 sec | 	solution = 1.00000 | 	eqconst = -0.00000
scipy_eigh_evd      :     3 sec | 	solution = 1.00000 | 	eqconst = 0.00000
scipy_eigh_evr_ss   :     2 sec | 	solution = 1.00000 | 	eqconst = 0.00000
scipy_eigh_evx_ss   :     2 sec | 	solution = 1.00000 | 	eqconst = 0.00000
scipy_eigh_gv       :     3 sec | 	solution = 1.00000 | 	eqconst = -0.00000
scipy_eigh_gvd      :     4 sec | 	solution = 1.00000 | 	eqcons

TODO: Plot the speed differences in a plot
TODO: Plot the outputs